In [1]:
import seaborn

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import linear as lr

In [2]:
%env CUDA_VISIBLE_DEVICES=[]

env: CUDA_VISIBLE_DEVICES=[]


# Linear Regression (continious target)

In [23]:
BATCH_SIZE = 100
TRAINING_EPOCHS = 500

In [24]:
data = lr.load_linear_data(500)
my_dataset = lr.load_dataset(data[0].shape[0])

In [25]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
cost_history = []

In [26]:
my_dataset.train.p \
    .load(data) \
    .preprocess_linear_data() \
    .train_linear(sess, cost_history) \
    .run(BATCH_SIZE, shuffle=True, n_epochs=TRAINING_EPOCHS)

## Оценка качества

In [31]:
mse = []
y_pred = np.zeros([len(my_dataset.test.indices), 1])
y_true = np.zeros([len(my_dataset.test.indices), 1])
x_features = np.zeros([len(my_dataset.test.indices), 13])

In [32]:
test_batch = my_dataset.test.p \
    .load(data) \
    .preprocess_linear_data() \
    .test_linear(sess, y_true, y_pred, mse, x_features) \
    .run(len(my_dataset.test.indices))

In [33]:
print ('Variance ratio: %.2f' % (np.var(y_pred, ddof=1) /np.var(y_true, ddof=1)))

Variance ratio: 0.83


In [34]:
print ('The model\'s error is distibuted in the interval: {0:.2f} $ ± {1:.2f} $'.format(np.mean(np.abs(y_pred - y_true)),  3*np.std(np.abs(y_pred - y_true))))

The model's error is distibuted in the interval: 0.15 $ ± 0.22 $


In [35]:
 print ('The model\'s error with respect to data\'s mean is: {0:.2f}%'.format(np.mean(np.abs(y_pred - y_true)/y_true)*100))

The model's error with respect to data's mean is: 0.03%


# Logistic Regression (Binary target)

In [36]:
data = lr.load_random_data(blobs=False)
my_dataset = lr.load_dataset(data[0].shape[0])

In [37]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
cost_history = []
acc_history = []

In [38]:
my_dataset.train.p \
    .load(data) \
    .preprocess_binary_data() \
    .train_logistic(sess, cost_history, acc_history) \
    .run(BATCH_SIZE, shuffle=True, n_epochs=TRAINING_EPOCHS)

## Оценка качества

In [39]:
acc = []
test_batch = my_dataset.test.p \
    .load(data) \
    .preprocess_binary_data() \
    .test_logistic(sess, acc) \
    .run(len(my_dataset.test.indices))

In [40]:
print("ACCURACY: %.0f%%" % (100.0 * acc[0]))

ACCURACY: 95%


# Poisson Regression (discret target)

In [41]:
data = lr.load_poisson_data()
my_dataset = lr.load_dataset(data[1].shape[0])

In [42]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
cost_history = []

In [43]:
my_dataset.train.p \
    .load(data[1:]) \
    .train_poisson(sess, cost_history) \
    .run(BATCH_SIZE, shuffle=True, n_epochs=TRAINING_EPOCHS)

In [44]:
logit = np.zeros([len(my_dataset.test.indices), 1])
y_true = np.zeros([len(my_dataset.test.indices), 1])
weights = np.zeros(data[0].shape)

In [45]:
test_batch = my_dataset.test.p.load(data[1:]).test_poisson(sess, y_true, logit, weights).run(len(my_dataset.test.indices))

## Оценка качества

In [46]:
lmbd = np.dot(data[1], data[0])
print ('Variance ratio: %.2f' % (np.var(logit, ddof=1) / np.var(lmbd, ddof=1)))

Variance ratio: 0.83


In [47]:
 print ('The model\'s error with respect to data\'s mean is: {0:.2f}%'.format(np.mean(np.abs(np.exp(logit) - y_true))/np.mean(y_true)*100))

The model's error with respect to data's mean is: 18.19%
